# 导库

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import json
import random

from utils import extend_maps, prepocess_data_for_lstmcrf, get_device, seed_everything
from models.util import tensorized, sort_by_lengths, cal_loss, cal_lstm_crf_loss
from models.bilstm_crf import BiLSTM_CRF

from ModelHandler import ModelHandler

In [2]:
device_all, n_gpu=get_device()

# 使用第一块 gpu
device = device_all
print(device, n_gpu)

device = "cpu"

device is cpu, not recommend
[] 0


In [3]:
# 设置随机种子
seed_everything()
random.randint(1,10)


3

# 数据预处理

In [4]:
df_train_data = pd.read_csv("./ResumeNER/df_train_data.csv", index_col=0)
df_dev_data = pd.read_csv("./ResumeNER/df_dev_data.csv", index_col=0)
df_test_data = pd.read_csv("./ResumeNER/df_test_data.csv", index_col=0)

print(df_train_data.shape, df_dev_data.shape, df_dev_data.shape)
df_train_data.head()

(3821, 4) (463, 4) (463, 4)


,words,words_index,tags,tags_index
0,"['高', '勇', '：', '男', '，', '中', '国', '国', '籍', ...","[0, 1, 2, 3, 4, 5, 6, 6, 7, 4, 8, 9, 10, 11, 1...","['B-NAME', 'E-NAME', 'O', 'O', 'O', 'B-CONT', ...","[0, 1, 2, 2, 2, 3, 4, 4, 5, 2, 2, 2, 2, 2, 2, ..."
1,"['1', '9', '6', '6', '年', '出', '生', '，', '汉', ...","[14, 15, 16, 16, 17, 18, 19, 4, 20, 21, 4, 5, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-RA...","[2, 2, 2, 2, 2, 2, 2, 2, 6, 7, 2, 8, 9, 9, 10,..."
2,"['2', '0', '0', '7', '年', '1', '0', '月', '至', ...","[55, 56, 56, 57, 17, 14, 56, 58, 59, 60, 61, 6...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 14, 15, 15, ..."
3,"['2', '0', '0', '7', '年', '1', '2', '月', '至', ...","[55, 56, 56, 57, 17, 14, 55, 58, 59, 55, 56, 1...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,"['2', '0', '1', '3', '年', '2', '月', '至', '今', ...","[55, 56, 14, 69, 17, 55, 58, 59, 60, 61, 78, 7...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 14, 15, 15, 15,..."


In [5]:
train_word_lists = [eval(i) for i in df_train_data['words'].tolist()]
train_tag_lists = [eval(i) for i in df_train_data['tags'].tolist()]

dev_word_lists = [eval(i) for i in df_dev_data['words'].tolist()]
dev_tag_lists = [eval(i) for i in df_dev_data['tags'].tolist()]

test_word_lists = [eval(i) for i in df_test_data['words'].tolist()]
test_tag_lists = [eval(i) for i in df_test_data['tags'].tolist()]


In [8]:
with open('./ResumeNER/word2id.json', 'r') as f:
    word2id = json.load(f)
with open('./ResumeNER/tag2id.json', 'r') as f:
    tag2id = json.load(f)
print(len(word2id), len(tag2id))
    
# （LSTM 需要使用 unk、pad）增加 unk，pad
# 为了让转移矩阵更具鲁棒性，添加 <start>，<end> 等字段
word2id, tag2id = extend_maps(word2id, tag2id, for_crf=True)

# 给每个句子添加 <end> 字段，测试数据不用加
train_word_lists, train_tag_lists = prepocess_data_for_lstmcrf(train_word_lists, train_tag_lists)
dev_word_lists, dev_tag_lists = prepocess_data_for_lstmcrf(dev_word_lists, dev_tag_lists)
test_word_lists, test_tag_lists = prepocess_data_for_lstmcrf(test_word_lists, test_tag_lists, test=True)

# 给句子进行排序，按照句子长度从小到大排序
train_word_lists, train_tag_lists, _ = sort_by_lengths(train_word_lists, train_tag_lists)
dev_word_lists, dev_tag_lists, _ = sort_by_lengths(dev_word_lists, dev_tag_lists)
test_word_lists, test_tag_lists, indices = sort_by_lengths(test_word_lists, test_tag_lists)

print(len(train_word_lists), len(train_tag_lists))
print(len(dev_word_lists), len(dev_tag_lists))
print(len(test_word_lists), len(test_tag_lists))

1902 28
3821 3821
463 463
477 477


# 模型单独测试

In [9]:
print(len(train_word_lists))
batch_sents = train_word_lists[:1]
batch_tags = train_tag_lists[:1]

emb_size, hidden_size = 128, 128
vocab_size = len(word2id)
out_size = len(tag2id)
print("out_size", out_size)
model = BiLSTM_CRF(vocab_size, emb_size, hidden_size, out_size).to(device)

model.train()
# 准备数据，变成 tesnor 数据
tensorized_sents, lengths = tensorized(batch_sents, word2id)
print(tensorized_sents.shape)
tensorized_sents = tensorized_sents.to(device)

targets, lengths = tensorized(batch_tags, tag2id)
print(targets.shape)
targets = targets.to(device)

# forward
scores = model(tensorized_sents, lengths).to(device)
print(scores.shape)
# 计算损失 更新参数
# self.optimizer.zero_grad()
loss = cal_lstm_crf_loss(scores, targets, tag2id).to(device)
print(loss.item())
# loss.backward()
# self.optimizer.step()

3821
out_size 32
torch.Size([1, 181])
torch.Size([1, 181])
torch.Size([1, 181, 32, 32])
616.3934936523438


# bilstm_crf 模型

In [ ]:
emb_size, hidden_size = 128, 128
vocab_size = len(word2id)
out_size = len(tag2id)
print("out_size", out_size)

model = BiLSTM_CRF(vocab_size, emb_size, hidden_size, out_size)



# 模型训练

In [10]:
# %%time
# seed_everything()

batch_size = 2**5

modelHandlerParams = {}
modelHandlerParams['epoch_nums'] = 1000000
modelHandlerParams['batch_size'] = batch_size
modelHandlerParams['device'] = device
modelHandlerParams['model'] = model
modelHandlerParams['word2id'] = word2id
modelHandlerParams['tag2id'] = tag2id
modelHandler = ModelHandler(modelHandlerParams)

model.to(device)

model_path = "./models/blstm_crf.pickle"

# crf 损失函数
loss_fn = cal_lstm_crf_loss
# 调参地方，分别调整为0.1,0.01,0.001，最优为0.01
optimizer = optim.Adam(model.parameters(), lr=0.01,
                       weight_decay=0.0001) # lr sets the learning rate of the optimizer

modelHandler.fit(X_train=train_word_lists, y_train=train_tag_lists, loss_fn=loss_fn, model_path = model_path,
                 optimizer=optimizer, eval_set = (dev_word_lists, dev_tag_lists), early_stopping_rounds=10, verbose=2)

epoch: 0
train loss: 22.425802663594734
*****Checking accuracy on validation set*****
batchNum: 14
val_loss: 6.7205063956124445
epoch: 1
train loss: 3.9375369929465927
*****Checking accuracy on validation set*****
batchNum: 14
val_loss: 3.7137156895228793
epoch: 2
train loss: 1.9068144469701944
*****Checking accuracy on validation set*****
batchNum: 14
val_loss: 3.5055945260184154
epoch: 3
train loss: 1.1473891955463826
*****Checking accuracy on validation set*****
batchNum: 14
val_loss: 2.8882337297712053
epoch: 4
train loss: 0.7612767580176601
*****Checking accuracy on validation set*****
batchNum: 14
val_loss: 2.9337141854422435
epoch: 5
train loss: 0.40826845569770875
*****Checking accuracy on validation set*****
batchNum: 14
val_loss: 3.069084712437221
epoch: 6
train loss: 0.2618102386218159
*****Checking accuracy on validation set*****
batchNum: 14
val_loss: 3.024627140590123
epoch: 7
train loss: 0.4107361480969341
*****Checking accuracy on validation set*****
batchNum: 14
val_lo

# 模型测试

In [11]:
pred_tag_lists, test_tag_lists = modelHandler.test(test_word_lists, test_tag_lists, indices)

from Metrics import Metrics
metrics = Metrics(test_tag_lists, pred_tag_lists)
metrics.report_scores()
metrics.report_confusion_matrix()

           precision    recall  f1-score   support
    M-ORG     0.3190    0.3202    0.3196      4325
   E-NAME     0.0177    0.0179    0.0178       112
    E-EDU     0.0089    0.0089    0.0089       112
    E-PRO     0.0000    0.0000    0.0000        33
   M-NAME     0.0100    0.0122    0.0110        82
    M-LOC     0.0000    0.0000    0.0000        21
  E-TITLE     0.0379    0.0376    0.0377       772
   B-CONT     0.0357    0.0357    0.0357        28
   M-CONT     0.0377    0.0377    0.0377        53
    E-LOC     0.0000    0.0000    0.0000         6
    B-EDU     0.0179    0.0179    0.0179       112
    B-PRO     0.0000    0.0000    0.0000        33
  B-TITLE     0.0616    0.0609    0.0612       772
  M-TITLE     0.1377    0.1296    0.1335      1922
   B-RACE     0.0714    0.0714    0.0714        14
    M-EDU     0.0292    0.0279    0.0286       179
    M-PRO     0.0000    0.0000    0.0000        68
    E-ORG     0.0432    0.0434    0.0433       553
   E-RACE     0.0714    0.0714 